In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import box


In [3]:
# EXPOSICIÓN A MICROPLÁSTICOS

noaa = gpd.read_file("../data/GeoDataFrame/gdf_microplastics.gpkg")
noaa = noaa.to_crs("EPSG:4326")

In [4]:
noaa_water = noaa[noaa['unit'] == 'pieces/m3']

In [6]:
# CREAR GRILLA (La grilla se define por el dataset que mide la EXPOSICIÓN.)

GRID_SIZE = 0.5  # grados
minx, miny, maxx, maxy = noaa.total_bounds

xs = np.arange(minx, maxx, GRID_SIZE)
ys = np.arange(miny, maxy, GRID_SIZE)

cells = []
ids = []

for i, x in enumerate(xs):
    for j, y in enumerate(ys):
        cells.append(box(x, y, x+GRID_SIZE, y+GRID_SIZE))
        ids.append(f"cell_{i}_{j}")

grid = gpd.GeoDataFrame(
    {"grid_id": ids},
    geometry=cells,
    crs="EPSG:4326"
)


In [7]:
# ASIGNAR PUNTOS DE NOAA A CELDAS DE GRID

noaa_in_grid = gpd.sjoin(noaa_water, grid, predicate="within")

In [10]:
# AGREGAR POR CELDA

noaa_features = (
    noaa_in_grid
    .groupby("grid_id")
    .agg(
        mp_pieces_m3=("microplastics_measurement", "max")
    )
    .reset_index()
)

noaa_features


,grid_id,mp_pieces_m3
0,cell_0_184,0.440000
1,cell_0_211,1.522606
2,cell_0_212,0.151503
3,cell_0_213,0.910641
4,cell_0_214,0.651202
...,...,...
5402,cell_99_222,0.246531
5403,cell_99_223,0.941444
5404,cell_99_224,0.239722
5405,cell_99_227,0.580413


In [13]:
grid = grid.merge(noaa_features, on="grid_id", how="left")


In [14]:
tomex = pd.read_csv("../data/Raw/ToMEx_sp_ml.csv")

In [15]:
eco_effect = tomex[
    tomex["Effect"] == "Yes"
]


In [16]:
eco_tox_threshold = eco_effect["Unaligned Dose Values"].median()
eco_tox_threshold

np.float64(62.5)

In [19]:
log_eco_tox_threshold = np.log10(eco_tox_threshold)
log_eco_tox_threshold

np.float64(1.7958800173440752)

In [17]:
# Conversion unidades de NOAA
grid["mp_particles_ml"] = grid["mp_pieces_m3"] / 1_000_000

In [20]:
import numpy as np

grid["log_mp_particles_ml"] = np.log10(
    grid["mp_particles_ml"] + 1e-12
)